# Classic semantic segmentation network
For this step, you have to train a classic segmentation network (**DeepLabV2** [2]) on the Cityscapes dataset.
- Dataset: **Cityscapes** [5]
- Training epochs: 50
- Training resolution (Cityscapes): 1024x512
- Test resolution (Cityscapes): 1024x512
- Backbone: **R101** (pre-trained on ImageNet) [2]
- Semantic classes: 19
- Metrics: Mean Intersection over Union (**mIoU**) [read this to understand the metrics], **latency**, **FLOPs**, number of **parameters**.


In [1]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 880.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=f925baa3f0f47c9b8e9dc9e56cf36b9246a32459c0fb49241c84d580c8c6dfe7
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=05907c252bd703aacc7aab3ce4753e37db9bc921fdbb9f9be547bdd603b680c9
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [2]:
# WANDB
!pip install -q wandb

# 0 - Import libraries

In [3]:
import wandb

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

import os
import zipfile
import numpy as np
import time
from PIL import Image

from fvcore.nn import FlopCountAnalysis, flop_count_table

import warnings
warnings.filterwarnings(action='ignore')

# 1 - Start WanDB

In [4]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 2 - Model Pipeline

In [5]:
def model_pipeline(hyperparameters=None):

    # tell wandb to get started
    with wandb.init(project="MLDL-step2a", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, val_loader, criterion, optimizer = make(config)
        #print(model)

        # and use them to train the model
        train(model, train_loader, criterion, optimizer, config)

        # and test its final performance
        val(model, val_loader)

    return model

# 3 - CityScapes

In [6]:
def make(config):
    # Make the data
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size,train=True)
    test_loader = make_loader(test, batch_size=config.batch_size,train=False)

    # Make the model (DeepLabV2 with ResNet-101 backbone)
    pretrain_model_path = '/kaggle/input/deeplab_v2_model/pytorch/model_weight/1/deeplab_resnet_pretrained_imagenet.pth'
    model = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path=pretrain_model_path).cuda()

    # Make the loss and optimizer
    optimizer = optim.SGD(model.parameters(), 
                          lr=config.learning_rate, 
                          momentum=config.momentum, 
                          weight_decay=config.weight_decay)
    
    criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
    
    return model, train_loader, test_loader, criterion, optimizer

In [7]:
# Define transforms for preprocessing
image_transform = transforms.Compose([
        transforms.Resize((512,1024)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

label_transform = transforms.Compose([
        transforms.Resize((512,1024)),
    ])

root_dir ='/kaggle/input/cityscapes/Cityscapes/Cityspaces'

def get_data(train=True):
    if train == True:
        # train dataset
        dataset = CityScapes(root_dir=root_dir, split='train', image_transform=image_transform, label_transform=label_transform)
    else:
        # test dataset
        dataset = CityScapes(root_dir=root_dir, split='val', image_transform=image_transform, label_transform=label_transform)
    
    return dataset


def make_loader(dataset, batch_size=8, train=True):
    if train == True:
        # train dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
    else:
        # test dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False,drop_last=True)
    
    return dataloader

In [8]:
class CityScapes(Dataset):
    def __init__(self, root_dir, split='train', image_transform=None, label_transform=None):
        super(CityScapes, self).__init__()
        """
        Args:
            root_dir (string): Directory with all the images and annotations.
            split (string): 'train' or 'val'.
            transform (callable, optional): Optional transform to be applied on a sample.
        """

        self.root_dir = root_dir
        self.split = split
        self.image_transform = image_transform
        self.label_transform = label_transform

        # Get the image and label directories
        self.image_dir = os.path.join(root_dir, 'images', split)
        self.label_dir = os.path.join(root_dir, 'gtFine', split)

        # Get a list of all image files
        self.image_files = []
        for city_dir in os.listdir(self.image_dir):
            city_image_dir = os.path.join(self.image_dir, city_dir)
            self.image_files.extend([os.path.join(city_image_dir, f) for f in os.listdir(city_image_dir) if f.endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # Get the corresponding label image path
        label_name = img_name.replace('images', 'gtFine').replace('_leftImg8bit', '_gtFine_labelTrainIds')

        # Load image and label
        image = Image.open(img_name).convert('RGB')
        label = Image.open(label_name).convert('L')

        #print(np.unique(np.array(label).reshape(np.array(label).shape[0] * np.array(label).shape[1])).size)

        if self.image_transform:
            image = self.image_transform(image)
        if self.label_transform:
            label = self.label_transform(label)

        #print(np.unique(np.array(label).reshape(np.array(label).shape[0] * np.array(label).shape[1])).size)

        label = torch.Tensor(np.array(label))

        return image, label

# 4 - DeepLabV2

In [9]:
affine_par = True


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        padding = dilation
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=padding, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn2.parameters():
            i.requires_grad = False
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4, affine=affine_par)
        for i in self.bn3.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out


class ClassifierModule(nn.Module):
    def __init__(self, inplanes, dilation_series, padding_series, num_classes):
        super(ClassifierModule, self).__init__()
        self.conv2d_list = nn.ModuleList()
        for dilation, padding in zip(dilation_series, padding_series):
            self.conv2d_list.append(
                nn.Conv2d(inplanes, num_classes, kernel_size=3, stride=1, padding=padding,
                          dilation=dilation, bias=True))

        for m in self.conv2d_list:
            m.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.conv2d_list[0](x)
        for i in range(len(self.conv2d_list) - 1):
            out += self.conv2d_list[i + 1](x)
        return out


class ResNetMulti(nn.Module):
    def __init__(self, block, layers, num_classes):
        self.inplanes = 64
        super(ResNetMulti, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)  # change
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4)
        self.layer6 = ClassifierModule(2048, [6, 12, 18, 24], [6, 12, 18, 24], num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if (stride != 1
                or self.inplanes != planes * block.expansion
                or dilation == 2
                or dilation == 4):
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, affine=affine_par))
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        _, _, H, W = x.size()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer6(x)

        x = torch.nn.functional.interpolate(x, size=(H, W), mode='bilinear')

        if self.training == True:
            return x, None, None

        return x

    def get_1x_lr_params_no_scale(self):
        """
        This generator returns all the parameters of the net except for
        the last classification layer. Note that for each batchnorm layer,
        requires_grad is set to False in deeplab_resnet.py, therefore this function does not return
        any batchnorm parameter
        """
        b = []

        b.append(self.conv1)
        b.append(self.bn1)
        b.append(self.layer1)
        b.append(self.layer2)
        b.append(self.layer3)
        b.append(self.layer4)

        for i in range(len(b)):
            for j in b[i].modules():
                jj = 0
                for k in j.parameters():
                    jj += 1
                    if k.requires_grad:
                        yield k

    def get_10x_lr_params(self):
        """
        This generator returns all the parameters for the last layer of the net,
        which does the classification of pixel into classes
        """
        b = []
        if self.multi_level:
            b.append(self.layer5.parameters())
        b.append(self.layer6.parameters())

        for j in range(len(b)):
            for i in b[j]:
                yield i

    def optim_parameters(self, lr):
        return [{'params': self.get_1x_lr_params_no_scale(), 'lr': lr},
                {'params': self.get_10x_lr_params(), 'lr': 10 * lr}]


def get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path='DeepLab_resnet_pretrained_imagenet.pth'):
    model = ResNetMulti(Bottleneck, [3, 4, 23, 3], num_classes)

    # Pretraining loading
    if pretrain:
        print('Deeplab pretraining loading...')
        saved_state_dict = torch.load(pretrain_model_path)

        new_params = model.state_dict().copy()
        for i in saved_state_dict:
            i_parts = i.split('.')
            new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
        model.load_state_dict(new_params, strict=False)

    return model

# 5 - Training

In [10]:
def train(model, dataloader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)
    
    for epoch in range(config.epochs):
        running_loss = 0.0
        total_mIOU = 0
        total_images = 0
        
        for _, (inputs, targets) in enumerate(dataloader):

            inputs, targets = inputs.cuda(), id_processing(targets).cuda()
        
            outputs = model(inputs)

            loss = criterion(outputs[0], targets)

            # Backprpagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs[0].max(1)

            running_mIOU = mean_iou(outputs[0].size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)
                
        train_loss = running_loss / len(dataloader)
        mIOU = total_mIOU/total_images
        
        train_log(train_loss, mIOU, epoch)

In [11]:
def id_processing(targets):
    targets = targets.cuda()
    
    # Define valid indices
    valid_indices = torch.tensor(list(range(19)) + [255]).to(targets.device)

    # Replace all IDs not in valid_indices with 255
    processed_targets = torch.where(torch.isin(targets, valid_indices), targets, torch.tensor(255, device=targets.device))

    return processed_targets.long()

In [12]:
def mean_iou(num_classes, pred, target):
    mIOU = 0
    for i in range(len(pred)):
        hist = fast_hist(target[i].cpu().numpy(),pred[i].cpu().numpy(), num_classes)
        IOU = per_class_iou(hist)
        mIOU = mIOU + sum(IOU)/num_classes
    return mIOU

def fast_hist(a, b, n):
    """
    a and b are predict and mask respectively
    n is the number of classes
    """
    k = (a >= 0) & (a < n) #assign True if the value is in the range between 0 and 18 (class labels)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape((n, n))

def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

In [13]:
def train_log(loss, mIOU, epoch):
    #wandb.log({"epoch": epoch, "loss": loss, "mIOU":mIOU})
    wandb.log({"loss": loss, "mIOU":mIOU}, step= epoch)
    print(f"----------------------------------")
    print(f"Loss after {epoch} epochs: {loss:.3f}")
    print(f"mIOU after {epoch} epochs: {mIOU:.3f}%")

# 6 - Validation

In [14]:
# Validation method
def val(model, dataloader):
    model.eval()
    total_mIOU = 0
    total_images = 0
    latency_list = []
    FPS_list = []
    
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), id_processing(targets).cuda()
            
            start = time.time() # Record start time
            outputs = model(inputs)
            end = time.time() # Record end time

            # Calculate latency for this iteration
            latency_i = end - start
            latency_list.append(latency_i)

            # Calculate FPS for this iteration
            FPS_i = 1 / latency_i
            FPS_list.append(FPS_i)

            _, predicted = outputs.max(1)
            
            running_mIOU = mean_iou(outputs.size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)
        
    mIOU = total_mIOU/total_images
    latency = np.sum(latency_list) / len(latency_list)
    test_FPS = np.sum(FPS_list) / len(FPS_list)
    
    # compute flops and #param
    image, _ = next(iter(dataloader))
    height, width = image.shape[2], image.shape[3]
    zero_image = torch.zeros((1, 3, height, width))
    flops = FlopCountAnalysis(model, zero_image.cuda())
    print(flop_count_table(flops))

    print(f'\n\nmIoU: {(mIOU*100):.3f}%, Latency: {latency:.3f}, FPS: {test_FPS:.3f}')

    wandb.log({"mIOU":mIOU,"latency":latency,"FPS":test_FPS})

# 7 - Hyperparameter Sweeps using WanDB

In [21]:
sweep_config= {
    'name': 'DeepLabV2-sweep',
    'metric': {'name': 'loss', 'goal': 'minimize'}, # the goal is maximize the accuracy
    'method': 'random', # test all possible combinations of the hyperparameters
    'parameters': {
        'epochs': {'values': [5]},        
        'learning_rate': {'values': [0.1, 0.001, 0.0001]}, # 2 parameters to optimize during the sweep
        'batch_size': {'values': [2, 4, 8]},
        'momentum': {'values': [0.9]},
        'weight_decay': {'values': [5e-4]}
    }
}

In [22]:
sweep_id = wandb.sweep(sweep=sweep_config, project="MLDL-step2a")

Create sweep with ID: 0bsj16g0
Sweep URL: https://wandb.ai/polito-tmazzarini/MLDL-step2a/sweeps/0bsj16g0


In [23]:
if torch.cuda.is_available():
    print("Start hyperparameter sweeps\n")
    wandb.agent(sweep_id, function=model_pipeline, count=5)
else:
    print("CUDA is Not available")

Start hyperparameter sweeps



wandb: Agent Starting Run: ebq7xakp with config:
wandb: 	batch_size: 4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.1
wandb: 	momentum: 0.9
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: tommasomazzarini2001 (polito-tmazzarini). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 4.019
mIOU after 1 epochs: 0.057%
----------------------------------
Loss after 2 epochs: 1.624
mIOU after 2 epochs: 0.073%
----------------------------------
Loss after 3 epochs: 1.351
mIOU after 3 epochs: 0.093%
----------------------------------
Loss after 4 epochs: 1.222
mIOU after 4 epochs: 0.103%
----------------------------------
Loss after 5 epochs: 1.140
mIOU after 5 epochs: 0.113%
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 95.816G    |
|  conv1                         |  9.408K                |  0.308G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  4.194M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.

FPS,▁
latency,▁
loss,█▂▂▁▁
mIOU,▁▃▆█▆
FPS,63.34041
latency,0.13987
loss,1.14011
mIOU,0.08854


wandb: Agent Starting Run: 1of7cgqi with config:
wandb: 	batch_size: 2
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 1.316
mIOU after 1 epochs: 0.146%
----------------------------------
Loss after 2 epochs: 0.877
mIOU after 2 epochs: 0.184%
----------------------------------
Loss after 3 epochs: 0.748
mIOU after 3 epochs: 0.199%
----------------------------------
Loss after 4 epochs: 0.666
mIOU after 4 epochs: 0.210%
----------------------------------
Loss after 5 epochs: 0.616
mIOU after 5 epochs: 0.216%
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 95.816G    |
|  conv1                         |  9.408K                |  0.308G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  4.194M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.

FPS,▁
latency,▁
loss,█▄▂▁▁
mIOU,▁▅▇█▇
FPS,62.71103
latency,0.13693
loss,0.61631
mIOU,0.20096


wandb: Agent Starting Run: v24cez3b with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.9
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 0.982
mIOU after 1 epochs: 0.184%
----------------------------------
Loss after 2 epochs: 0.542
mIOU after 2 epochs: 0.230%
----------------------------------
Loss after 3 epochs: 0.472
mIOU after 3 epochs: 0.245%
----------------------------------
Loss after 4 epochs: 0.433
mIOU after 4 epochs: 0.254%
----------------------------------
Loss after 5 epochs: 0.408
mIOU after 5 epochs: 0.261%
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 95.816G    |
|  conv1                         |  9.408K                |  0.308G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  4.194M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.

FPS,▁
latency,▁
loss,█▃▂▁▁
mIOU,▁▆▇█▆
FPS,64.52771
latency,0.14766
loss,0.40814
mIOU,0.23719


# 8 - Build final model

In [15]:
# best configuration (TO CONFIGURE)
config = dict(
    epochs=50,
    batch_size=2,
    learning_rate=0.001,
    momentum=0.9,
    weight_decay=5e-4,
    architecture="DeepLabV2")

In [ ]:
if torch.cuda.is_available():
    print("Building the model with the best configuration")
    # Build, train and analyze the model with the pipeline
    model = model_pipeline(config)
else:
    print("CUDA is Not available")

wandb: Currently logged in as: tommasomazzarini2001 (polito-tmazzarini). Use `wandb login --relogin` to force relogin


Building the model with the best configuration


Deeplab pretraining loading...
----------------------------------
Loss after 0 epochs: 0.557
mIOU after 0 epochs: 0.255%
----------------------------------
Loss after 1 epochs: 0.314
mIOU after 1 epochs: 0.303%
----------------------------------
Loss after 2 epochs: 0.271
mIOU after 2 epochs: 0.318%
----------------------------------
Loss after 3 epochs: 0.247
mIOU after 3 epochs: 0.327%
----------------------------------
Loss after 4 epochs: 0.231
mIOU after 4 epochs: 0.335%
----------------------------------
Loss after 5 epochs: 0.220
mIOU after 5 epochs: 0.340%
----------------------------------
Loss after 6 epochs: 0.210
mIOU after 6 epochs: 0.345%
----------------------------------
Loss after 7 epochs: 0.202
mIOU after 7 epochs: 0.350%
----------------------------------
Loss after 8 epochs: 0.197
mIOU after 8 epochs: 0.353%
----------------------------------
Loss after 9 epochs: 0.191
mIOU after 9 epochs: 0.357%
----------------------------------
Loss after 10 epochs: 0.186
mIOU a

In [1]:
wandb.finish()

NameError: name 'wandb' is not defined